# Example: SED Fitting with sedfitter (using the Richardson+ (2024) YSO models)
`sedfitter` is a Python port of the SED fitting tool described in [Robitaille+ (2007)](). It is also the primary interface for the [Robitaille+ (2006)](https://ui.adsabs.harvard.edu/abs/2006ApJS..167..256R/exportcitation) and [Robitaille (2017)](https://ui.adsabs.harvard.edu/abs/2017A%26A...600A..11R/abstract) (R17) sets of radiative transfer YSO models. A [new version](https://zenodo.org/records/10522816) of the latter set was recently released alongside [Richardson+ (2024)](https://ui.adsabs.harvard.edu/abs/2024ApJ...961..188R/abstract) (R24). This version expands the available information, but use of these models will likely require slight modifications to a `sedfitter`-based workflow in order to maintain functionality.

The purpose of this notebook is to synthesize aspects of the [sedfitter documentation](https://sedfitter.readthedocs.io/en/stable/) into a single demonstration of the workflow and basic functionality of the tool, as well as to provide additional information on how to work with the R24 version of the 2017 YSO models. It is current as of PR #(number).

(Disclaimer: No guarantees are made about the accuracy of the results, and users are responsible for ensuring that the results are reasonable and that they understand all the limitations inherent to SED fitting.)

In [1]:
import os
import glob

import numpy as np
from astropy import units as u

from sedfitter import (fit, plot, plot_params_1d, plot_params_2d,
                       write_parameters, write_parameter_ranges)
from sedfitter.extinction import Extinction

In [2]:
%rm -rf r24_example
%mkdir r24_example

## Setup
In order to fit to a set of models, the fitter needs to be provided with their directory (i.e. the folder which contains the model parameters and SEDs). The R24 model information are contained within directories named for several "model geometries", which are subsets of models with common features. (The downloadable model files unzip to `r+24_models/{geometry}`.) Table 2 of R17 contains a full accounting of the names and significances of these geometries. For the purposes of this notebook, we pick `spubhmi`, which is the most populous geometry. `spubhmi` models all have disks, rotationally flattened envelopes with internal holes, and bipolar cavities.

In [3]:
# Define path to models
r24_modeldir = '/blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2'
geometry = 'spubhmi'

The fitter must also be provided with an extinction law in order to accommodate extinction from material along the line of sight. The extinction law should be in opacity units ($cm^2 / g$ or equivalent). Handling of extinction is unchanged from the time of the R17 release; the `sedfitter` documentation contains more information.

In [4]:
# Read in extinction law. We read in columns 1 (the wavelength in microns) and
# 4 (the opacity in cm^2/g)
extinction = Extinction.from_file('kmh94.par', columns=[0, 3],
                                  wav_unit=u.micron, chi_unit=u.cm**2 / u.g)

Both the R17 and R24 sets of models have convolved their SEDs with the response profiles of several current or formerly active instruments often used to observe YSOs. One of the main differences between the 2017 and 2024 releases of the YSO models is in how their filters are named; the 2017 release named them solely by the filter itself, while the 2024 version incorporates [astroquery](https://astroquery.readthedocs.io/en/latest/) into its convolution procedure to retrieve filter profiles from the [SVO FPS](https://svo2.cab.inta-csic.es/theory/fps/) (Rodrigo+ 2012/Rodrigo & Solano 2020). Consequently, when fitting using the 2024 models, the convolved fluxes must be invoked using the name as formatted by the SVO FPS; generally, this naming convention is `{facility}/{instrument}.{filter}`. In this data, we have access to 2MASS's J/H/K filters and the Spitzer IRAC's I1-I4.

Convolution in `sedfitter` is changed in this PR to use `numpy.nansum` instead of `numpy.sum`. This prevents losing information when convolving with SEDs which have had parts removed due to S/N post-processing (see Sec. 4.2.4 of R17); previously, filters which had defined in-band SEDs would have undefined fluxes due to `nan` values elsewhere in the SED, as convolution is performed over the entire wavelength range.

In [5]:
# Define filters
filters = ['2MASS/2MASS.J', '2MASS/2MASS.H', '2MASS/2MASS.Ks', 
           'Spitzer/IRAC.I1', 'Spitzer/IRAC.I2', 
           'Spitzer/IRAC.I3', 'Spitzer/IRAC.I4']

The fitter must also be provided with the radius of each aperture in which the data is taken in order to fit to the correct data. Handling of providing apertures to the fitter is unchanged from the time of the R17 release; the `sedfitter` documentation contains more information.

In [6]:
# Define apertures
apertures = [3., 3., 3., 3., 3., 3., 3.] * u.arcsec

## Fitting
Now that we have set up the data and specified the models to fit to it, we can perform the fit:

In [7]:
# Run the fitting
fit('data_glimpse', filters, apertures, f'{r24_modeldir}/{geometry}',
    'r24_example/output.fitinfo',
    extinction_law=extinction,
    distance_range=[1., 2.] * u.kpc,
    av_range=[0., 40.])

 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  spubhmi
   Log[d] stepping     :  0.02
   Number of distances :  17

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

Data shape=(720000, 17, 7).  use_memmap=True
   Reading /blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2/spubhmi/convolved/2MASS/2MASS.J.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2/spubhmi/convolved/2MASS/2MASS.H.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2/spubhmi/convolved/2MASS/2MASS.Ks.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2/spubhmi/convolved/Spitzer/IRAC.I1.fits
   Reading /blue/adamginsburg/richardson.t/research/flux/r+24_models-1.2/spubhmi/convolved/Spitzer/IRAC.I2.fits


After fitting, you will have a set of `FitInfo` objects that contain the results of fitting every model. Oftentimes when working with these, it is desirable to limit the number of models being considered. Model limitation in `sedfitter` is accomplished through the `select_format` argument, which is a keyword argument in many functions (see documentation). `sedfitter` provides a number of ways to downselect fitted models. A general overview of the various selection formats is provided here and in the documentation.

* `('A', value)`: select all models (`value` is ignored).
* `('N', value)`: select a fixed number of fits given by `value`.
* `('C', value)`: select all models with a $\chi^2$ below a threshold given by `value`.
* `('D', value)`: select all models with a $\chi^2-\chi^2_{\rm best}$ value below a threshold given by `value`.
* `('E', value)`: select all models with a $\chi^2$ per datapoint below a threshold given by `value`.
* `('F', value)`: select all models with a $\chi^2-\chi^2_{\rm best}$ value per datapoint below a threshold given by `value`.

By default, functions in `sedfitter` will work with the best-fitting model (i.e. `('N', 1)`).

For this example, we will limit consideration to models which exhibit a $\chi^2-\chi^2_{\rm best}$-per-data-point $<$ 3.

In [8]:
select_format = ('F', 3)

## Working with the fit results
After performing the fit and selecting the set of models to work with, we can visualize the results of the fitting in a number of ways, which we demonstrate here. The documentation contains additional information on the keyword arguments.

### Plotting SEDs
Firstly, we can plot the data and our downselected set of SEDs together:

In [9]:
# Make SED plots
plot('r24_example/output.fitinfo', 'r24_example/plots_seds', plot_max=100, select_format=select_format)

![sed plot](example_images/sed.png)

### Plotting parameters
We can also plot histograms which compare the parameters of the set of SEDs we have chosen with `select_format` to the parameters of all models in the set we fit to. This particular histogram shows the luminosity of the central sources of the models, which is made explicit in R24.

In [10]:
# Make histograms of the source luminosity
plot_params_1d('r24_example/output.fitinfo', 'Source Luminosity', 
               output_dir='r24_example/plots_lum',
               log_x=True, select_format=select_format)

![lum histogram](example_images/lum.png)

We can repeat this for another new parameter: the "sphere masses" of the models. These track the amount of dust and gas mass contained within spherical regions around the central source. The sphere mass of models is one of several "array parameters" new to the R24 set, which require some additional handling and explanation. SEDs in R24 are defined in a series of apertures with different physical radii. These radii are 20 evenly log-spaced values between $10^2-10^6$ AU. The array parameters are quantities which, for one reason or another, vary with aperture. 

Consequently, functions in sedfitter which work with the table parameters now include an optional `aperture` keyword argument which picks all of the values in these array parameters consistent with a particular aperture. `aperture` accepts integer values; any integer which indexes a value within an array-like object of length 20 will work. In the following plot, we show sphere mass in the smallest available aperture:

In [11]:
# Make histograms of the sphere mass in the smallest aperture
plot_params_1d('r24_example/output.fitinfo', 'Sphere Masses',aperture=0,
               output_dir='r24_example/plots_sphmass_smallap',
               log_x=True, select_format=select_format)

![mass histogram in smallest aperture](example_images/mass_small.png)

That aperture has a radius of 100 AU, which is not really representative of the data. To instead find the appropriate aperture to use, we bring in some of the information provided to the fitter earlier. In this case, since we are allowing the distance to vary between 1-2 kpc, we will say that the source is at a distance of ~1.5 kpc for simplicity. Our 3-arcsecond aperture therefore corresponds to a physical radius of 4500 AU; we find the index of the aperture which is closest to that, which we will use as input to the `aperture` argument.

In [12]:
all_apertures = np.logspace(2,6,20) * u.AU
mid_distance_ap = 4500 * u.AU
ap_num = np.argmin(abs(mid_distance_ap - all_apertures))

Replotting the sphere mass histogram with this change:

In [13]:
# Make histograms of the sphere mass in the smallest aperture
plot_params_1d('r24_example/output.fitinfo', 'Sphere Masses',aperture=ap_num,
               output_dir='r24_example/plots_sphmass_fitap',
               log_x=True, select_format=select_format)

![mass histogram in best aperture](example_images/mass_fit.png)

We can also plot 2D histograms, where we compare the distributions of two parameters at once. In this case, we will plot the source luminosities and sphere masses together (taking sphere masses within the "right" aperture).

In [14]:
# Make 2-d plots of the source luminosity vs circumstellar mass, 
# in an aperture consistent with the size used for fitting
plot_params_2d('r24_example/output.fitinfo', 
               'Source Luminosity', 'Sphere Masses',
               output_dir='r24_example/plots_lum_sphmass_fitap',
               log_x=True, log_y=True, select_format=select_format,
               aperture=ap_num)

![2D luminosity/mass histogram](example_images/lum_mass.png)

### Creating parameter tables
Finally, we can write information about the model parameters in our downselected set out to files. This information can either be every parameter value for each model in the set, or it can be the range of values in each parameter (and best-fit parameter values) over the set. Array parameters are handled as above, where a single value from the array is written out based on the provided aperture index.

In [15]:
# Write out all models with a delta chi^2-chi_best^2 per datapoint < 3
write_parameters('r24_example/output.fitinfo', 'r24_example/parameters.txt',
                 select_format=select_format,
                 aperture=ap_num)

# Write out the min/max ranges corresponding to the above file
write_parameter_ranges('r24_example/output.fitinfo', 'r24_example/parameter_ranges.txt',
                       select_format=select_format,
                       aperture=ap_num)

/blue/adamginsburg/richardson.t/dev/sedfitter/sedfitter/write_parameters.py:102: UserWarning: Warning: converting a masked element to nan.
  fout.write('%10.3e ' % (vals[aperture]))
/blue/adamginsburg/richardson.t/dev/sedfitter/sedfitter/write_parameters.py:104: UserWarning: Warning: converting a masked element to nan.
  fout.write('%10.3e ' % (vals[0]))
/blue/adamginsburg/richardson.t/dev/sedfitter/sedfitter/write_parameter_ranges.py:147: UserWarning: Warning: converting a masked element to nan.
  fout.write('%10.3e %10.3e %10.3e ' % (np.nanmin(col), col[0], np.nanmax(col)))
